# Pandas and SQLAlchemy

Pandas has several functions to work with databases via sqlalchemy. 

This can be an easier way to get data from a database and to pass data into a database.

Let's start with loading the `world.sqlite` database we've been using.

In [14]:
#!/usr/bin/env python
from sqlalchemy import create_engine
from sqlalchemy import MetaData
from sqlalchemy import Table, Column
from sqlalchemy import Integer, String
from sqlalchemy import sql, select, join, desc

# Create a Engine object which is our handle into the database.
engine = create_engine('sqlite:////blue/bsc4452/share/Class_Files/data/world.sqlite')

# Connect to the database
conn = engine.connect()

# Read the metadata from the existing database.
#  Since the database already exists and has tables defined, we can create Python objects based on these automatically.
DBInfo=MetaData(engine)

The cell below is also from the [SQLAlchemy notebook](SQLAlchemy.ipynb) and creates the `city` object with the table metadata, creates the `query` variable. This cell shows how you would execute the query with SQLAlchemy, put the result into the `result` variable and then loops through that printing each row. The last line was added to show the the `query` is of type `<class 'sqlalchemy.sql.selectable.Select'>`, or in words a **SQLAlchemy Selectable**, one of the options for the `pandas.read_sql` function we'll look at next.

In [6]:
# Auto-create the city object based on the metadata read into the DBInfo.
city=Table('city', DBInfo, autoload=True)

# Mirroring this command: SELECT Id, Name, Population FROM city LIMIT 10;
query=select([city.c.ID, city.c.Name, city.c.Population]).limit(10)
result = conn.execute(query)
for row in result:
    print(row)
    
print(type(query))

(1, 'Kabul', 1780000)
(2, 'Qandahar', 237500)
(3, 'Herat', 186800)
(4, 'Mazar-e-Sharif', 127800)
(6, 'Rotterdam', 593321)
(7, 'Haag', 440900)
(8, 'Utrecht', 234323)
(9, 'Eindhoven', 201843)
(10, 'Tilburg', 193238)
(11, 'Groningen', 172701)
<class 'sqlalchemy.sql.selectable.Select'>


## Reading SQL query results into a Pandas data frame

Here's how we can put the result into a pandas data frame. The Selectable and connection are the required arguments for `read_sql`. In this example, we also add that the ID column should be used for the index of the data frame.

In [7]:
import pandas as pd

df=pd.read_sql(query, conn, index_col='ID')

df.head()

,Name,Population
ID,,
1,Kabul,1780000
2,Qandahar,237500
3,Herat,186800
4,Mazar-e-Sharif,127800
6,Rotterdam,593321


## Writing a Pandas data frame to a database

In the next example, we'll use the `shipping_tables.xlsx` table that we used in the [Reading poorly structured Excel files with Pandas notebook](Pandas_messy_data.ipynb).

First, we'll get the data into the dataframes

In [9]:
orders = pd.read_excel('shipping_tables.xlsx', header=1, usecols='B:F')
orders.head()

,order id,order date,state,priority,item_type
0,669165933,2019-01-03,MN,2-day,Baby Food
1,963881480,2019-01-04,WI,next-day,Cereal
2,341417157,2019-01-05,TX,2-day,Office Supplies
3,514321792,2019-01-06,CA,next-day,Office Supplies
4,115456712,2019-01-07,CA,2-day,Office Supplies


In [10]:
from openpyxl import load_workbook
wb = load_workbook(filename = 'shipping_tables.xlsx') 
sheet = wb['shipping_rates']
lookup_table = sheet.tables['ship_cost']
data = sheet[lookup_table.ref]
rows_list = []
for row in data:
    cols = []
    for col in row:
        cols.append(col.value)
    rows_list.append(cols)
ship_rates = pd.DataFrame(data=rows_list[1:], index=None, columns=rows_list[0])
ship_rates.head()

,item_type,priority,shipping_cost
0,Baby Food,2-day,5
1,Baby Food,next-day,7
2,Cereal,2-day,8
3,Cereal,next-day,11
4,Fruit,2-day,5


Now we have two Pandas data frames and can create a database and add the data.

In [25]:
# Create the database and establish the connection.
engine = create_engine('sqlite:///shipping.sqlite')
conn = engine.connect()

# Use the `to_sql` function to send the data frames to the database.
orders.to_sql('orders', conn)
ship_rates.to_sql('ship_rates', conn)

# Run a raw engine.execute with SQL to see that the data are there
# We could create objects, read the metadata, etc. but seems excessive...
engine.execute("SELECT * FROM orders").fetchall()

[(0, 669165933, '2019-01-03 00:00:00.000000', 'MN', '2-day', 'Baby Food'),
 (1, 963881480, '2019-01-04 00:00:00.000000', 'WI', 'next-day', 'Cereal'),
 (2, 341417157, '2019-01-05 00:00:00.000000', 'TX', '2-day', 'Office Supplies'),
 (3, 514321792, '2019-01-06 00:00:00.000000', 'CA', 'next-day', 'Office Supplies'),
 (4, 115456712, '2019-01-07 00:00:00.000000', 'CA', '2-day', 'Office Supplies'),
 (5, 547995746, '2019-01-08 00:00:00.000000', 'NY', 'next-day', 'Cereal'),
 (6, 135425221, '2019-01-09 00:00:00.000000', 'NY', 'next-day', 'Cereal'),
 (7, 871543967, '2019-01-10 00:00:00.000000', 'TX', 'next-day', 'Fruit'),
 (8, 770463311, '2019-01-11 00:00:00.000000', 'FL', '2-day', 'Baby Food')]